# Assignment 1: This is the title of the assignment

Authors:  
Lina Hagberg  
Ella Villför  

## Imports

In [ ]:
# It is recommended to start with general import statements
from utility_functions import *

## Load data

This section should load the raw dataset for the task.  
Remember to use relative paths to load any files in the notebook.

In [ ]:
# Always use comments in the code to document specific steps

## Task 1: This is the title of task 1

This section should contain the solution of task 1.

It is mandatory to maintain the headings for each task.  
OPTIONALLY, you can use one level down (###) to organize subsessions of the assignments.

Use markdown cells like this one to include:
- Discussion points.
- References to specific sources of code that you might have used to solve the assignment.
- General commentas and explanations about your solution.

In [ ]:
# Always use comments in the code to document specific steps

## Task 2: This is the title of task 2

This section should contain the solution of task 2.

## Results and Discussion

This section should contain:
- Results.
- Summary of best model performance:
    - Name of best model file as saved in /models.
    - Relevant scores such as: accuracy, precision, recall, F1-score, etc.
- Key discussion points.

In [ ]:
# Always use comments in the code to document specific steps